In [ ]:
OPEN_WEATHER_API_KEY = "" # https://openweathermap.org/api
MAPBOX_API_KEY = "" # https://www.mapbox.com

# Load Data

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

In [ ]:
df_left = pd.read_csv("./data/left.csv", low_memory=False)
df_right = pd.read_csv("./data/right.csv", low_memory=False)
df_inner = pd.read_csv("./data/inner.csv", low_memory=False)
df_outer = pd.read_csv("./data/outer.csv", low_memory=False)

In [ ]:
df_left.head()

In [ ]:
df_left.dtypes

In [ ]:
df_left.info()

In [ ]:
df_left.describe()

# Geolocation

In [ ]:
def plot_side_by_side(df_temp, df_humidity, title):
    fig = make_subplots(
        rows=2,
        cols=2,
        subplot_titles=(
            "Avg Temperature",
            "Max Temperature",
            "Avg Humidity",
            "Max Humidity",
        ),
    )

    # Average Temperature Plots
    fig.add_trace(
        go.Bar(
            x=df_temp["country"],
            y=df_temp["avg_weather_station_temperature"],
            name="Weather Station",
            legendgroup="group1",
            marker_color="#a56eff",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Bar(
            x=df_temp["country"],
            y=df_temp["avg_truck_temperature"],
            name="Truck Sensor",
            legendgroup="group1",
            marker_color="#005d5d",
        ),
        row=1,
        col=1,
    )

    # Maximum Temperature Plots
    fig.add_trace(
        go.Bar(
            x=df_temp["country"],
            y=df_temp["max_weather_station_temperature"],
            name="Weather Station",
            legendgroup="group1",
            showlegend=False,
            marker_color="#a56eff",
        ),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Bar(
            x=df_temp["country"],
            y=df_temp["max_truck_temperature"],
            name="Truck Sensor",
            legendgroup="group1",
            showlegend=False,
            marker_color="#005d5d",
        ),
        row=1,
        col=2,
    )

    # Average Humidity Plots
    fig.add_trace(
        go.Bar(
            x=df_humidity["country"],
            y=df_humidity["avg_weather_station_humidity"],
            name="Weather Station",
            legendgroup="group1",
            showlegend=False,
            marker_color="#a56eff",
        ),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Bar(
            x=df_humidity["country"],
            y=df_humidity["avg_truck_humidity"],
            name="Truck Sensor",
            legendgroup="group1",
            showlegend=False,
            marker_color="#005d5d",
        ),
        row=2,
        col=1,
    )

    # Maximum Humidity Plots
    fig.add_trace(
        go.Bar(
            x=df_humidity["country"],
            y=df_humidity["max_weather_station_humidity"],
            name="Weather Station",
            legendgroup="group1",
            showlegend=False,
            marker_color="#a56eff",
        ),
        row=2,
        col=2,
    )
    fig.add_trace(
        go.Bar(
            x=df_humidity["country"],
            y=df_humidity["max_truck_humidity"],
            name="Truck Sensor",
            legendgroup="group1",
            showlegend=False,
            marker_color="#005d5d",
        ),
        row=2,
        col=2,
    )

    # Add h_line to show the dangerous line and make it go across the entire plot
    # Dangerous Temp
    fig.add_hline(
        y=25,
        line_color="#9f1853",
        row=1,
        col=1,
        
    )
    fig.add_hline(
        y=25,
        line_color="#9f1853",
        row=1,
        col=2,
    )
    # Dangerous Humidity
    fig.add_hline(
        y=80,
        line_color="#9f1853",
        row=2,
        col=1,
    )
    fig.add_hline(
        y=80,
        line_color="#9f1853",
        row=2,
        col=2,
    )
  
    # Hack to add legend for dangerous line
    fig.add_trace(
        go.Scatter(
            x=df_temp["country"],
            y=[25] * len(df_temp["country"]),
            mode="lines",
            name="Dangerous",
            line=dict(color="#9f1853", width=2),
            legendgroup="group_dangerous_line",
            showlegend=True,
        ),
        row=1,
        col=1,
    )    

    # Update layout to make it transparent, figure size, and title text
    fig.update_layout(
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        height=800, width=1200, title_text=title
    )   

    return fig

In [ ]:
# Rename column to make distinction between weather station and truck sensor
df_left = df_left.rename(
    columns={
        "temp": "weather_station_temperature",
        "humidity": "weather_station_humidity",
    }
)

In [ ]:
df_temp_avg = (
    df_left.query("measurement == 'temperature'")
    .groupby(["country"])
    .agg(
        avg_weather_station_temperature=("weather_station_temperature", "mean"),
        avg_truck_temperature=("value", "mean"),
        max_weather_station_temperature=("weather_station_temperature", "max"),
        max_truck_temperature=("value", "max"),
    )
    .reset_index()
)
df_temp_avg

In [ ]:
df_humidity_avg = (
    df_left.query("measurement == 'humidity'")
    .groupby(["country"])
    .agg(
        avg_weather_station_humidity=("weather_station_humidity", "mean"),
        avg_truck_humidity=("value", "mean"),
        max_weather_station_humidity=("weather_station_humidity", "max"),
        max_truck_humidity=("value", "max"),
    )
    .reset_index()
)
df_humidity_avg

In [ ]:
plot_side_by_side(df_temp_avg, df_humidity_avg, "Average and Maximum Temperature and Humidity")

# Time

In [ ]:
# Set the timestamp column to datetime and create new columns for month, day, and hour
df_left["timestamp"] = pd.to_datetime(df_left["timestamp"])
df_left["timestamp_month"], df_left["timestamp_day"], df_left["timestamp_hour"] = (
    df_left["timestamp"].dt.month,
    df_left["timestamp"].dt.day,
    df_left["timestamp"].dt.hour,
)
# Replace month with month name
month_map = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December",
}
df_left["timestamp_month"] = df_left["timestamp_month"].map(month_map)
# Set as category & order by month
df_left["timestamp_month"] = pd.Categorical(
    df_left["timestamp_month"], categories=month_map.values(), ordered=True
)

# Replace day with day name
day_map = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}
df_left["timestamp_day"] = df_left["timestamp"].dt.dayofweek.map(day_map)
# Set as category & order by day
df_left["timestamp_day"] = pd.Categorical(
    df_left["timestamp_day"], categories=day_map.values(), ordered=True
)
# Set hour as am/pm
df_left["timestamp_hour"] = df_left["timestamp"].dt.strftime("%I %p")
# Order by hour
df_left["timestamp_hour"] = pd.Categorical(
    df_left["timestamp_hour"],
    categories=[
        "12 AM",
        "01 AM",
        "02 AM",
        "03 AM",
        "04 AM",
        "05 AM",
        "06 AM",
        "07 AM",
        "08 AM",
        "09 AM",
        "10 AM",
        "11 AM",
        "12 PM",
        "01 PM",
        "02 PM",
        "03 PM",
        "04 PM",
        "05 PM",
        "06 PM",
        "07 PM",
        "08 PM",
        "09 PM",
        "10 PM",
        "11 PM",
    ],
    ordered=True,
)

In [ ]:
df_humidity_avg_by_month = (
    df_left.query("measurement == 'humidity'")
    .groupby(["timestamp_month"])
    .agg(
        avg_weather_station_humidity=("weather_station_humidity", "mean"),
        avg_truck_humidity=("value", "mean"),
    )
    .reset_index()
)
df_humidity_avg_by_month

In [ ]:
# White to Dark Blue Color Scape
color_scale = [
    "#e5f6ff",
    "#bae6ff",
    "#82cfff",
    "#33b1ff",
    "#1192e8",
    "#0072c6",
    "#00539a",
    "#003a6d",
    "#012749",
    "#1c0f30",
]


# Diverging color scale
reds = list(
    reversed(
        [
            "#750e13",
            "#a2191f",
            "#da1e28",
            "#fa4d56",
            "#ff8389",
            "#ffb3b8",
            "#ffd7d9",
            "#fff1f1",
        ]
    )
)
blues = list(
    reversed(
        [
            "#e5f6ff",
            "#bae6ff",
            "#82cfff",
            "#33b1ff",
            "#1192e8",
            "#0072c6",
            "#00539a",
            "#003a6d",
        ]
    )
)
diverging_r_b = blues + reds

In [ ]:
# Average Truck Temperature
temperature_df = df_left.query("measurement == 'temperature'").sort_values(
    by=["timestamp_month"]
)

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        x=temperature_df["timestamp_month"],
        y=temperature_df["country"],
        z=temperature_df["value"],
        text=np.where(temperature_df["value"].isnull(), "", temperature_df["value"]),
        texttemplate="%{text}",
        colorscale=diverging_r_b,
        name="Temperature",
        hoverongaps=False,
        zmid=0,
        colorbar=dict(
            title="avg of Temperature (°C)",
            titleside="top",
        ),
    )
)


# Disable grids
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)


# Remove padding, set figure size, remove background color
fig.update_layout(
    margin=dict(t=0, b=0, l=0, r=0),
    height=600,
    width=800,
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
)

fig.show()

In [ ]:
# Average Truck Temperature
temperature_df = df_left.query("measurement == 'humidity'").sort_values(
    by=["timestamp_month"]
)

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        x=temperature_df["timestamp_month"],
        y=temperature_df["country"],
        z=temperature_df["value"],
        text=np.where(temperature_df["value"].isnull(), "", temperature_df["value"]),
        texttemplate="%{text}",
        colorscale=color_scale,
        name="Humidity",
        hoverongaps=False,
        zmin=0,
        colorbar=dict(
            title="avg of Humidity (%)",
            titleside="top",
        ),
    )
)

# Disable grids
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)


# Remove padding, set figure size, remove background color
fig.update_layout(
    margin=dict(t=0, b=0, l=0, r=0),
    height=600,
    width=800,
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
)

fig.show()

In [ ]:
fig = px.density_heatmap(
    df_left.query("measurement == 'temperature'").sort_values(
        by=["timestamp_hour", "timestamp_day"]
    ),
    x="timestamp_hour",
    y="timestamp_day",
    z="value",
    color_continuous_scale=reds,
    title="Average Truck Temperature per Day by Hour",
    histfunc="avg",
    labels={
        "timestamp_hour": "Hour",
        "timestamp_day": "Day",
        "value": "Temperature",
    },
    text_auto=".2f",
    template="plotly_white",
    height=600,
    width=1400,
)

# Transparent background
fig.update_layout(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
)

fig.show()

In [ ]:
fig = px.density_heatmap(
    df_left.query("measurement == 'humidity'").sort_values(
        by=["timestamp_hour", "timestamp_day"]
    ),
    x="timestamp_hour",
    y="timestamp_day",
    z="value",
    color_continuous_scale=color_scale,
    title="Average Truck Humidity per Day by Hour",
    histfunc="avg",
    labels={
        "timestamp_hour": "Hour",
        "timestamp_day": "Day",
        "value": "Humidity",
    },
    text_auto=".2f",
    template="plotly_white",
    height=600,
    width=1400,
)

# Transparent background
fig.update_layout(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
)


fig.show()

# Internal and External Weather/Climate

In [ ]:
df_temperature_avg_by_month = (
    df_left.query("measurement == 'temperature'")
    .groupby(["timestamp_month"])
    .agg(
        avg_truck_temperature=("value", "mean"),
    )
)

df_humidity_avg_by_month = (
    df_left.query("measurement == 'humidity'")
    .groupby(["timestamp_month"])
    .agg(
        avg_truck_humidity=("value", "mean"),
    )
)


df_climate_avg_by_month = df_left.groupby(["timestamp_month"]).agg(
    avg_rain=("rain", "mean"),
    avg_snow=("snow", "mean"),
    avg_cloudyness=("clouds", "mean"),
    avg_windspeed=("wind_speed", "mean"),
)

df_climary_summary = (
    pd.concat(
        [
            df_temperature_avg_by_month,
            df_humidity_avg_by_month,
            df_climate_avg_by_month,
        ],
        axis=1,
    )
    .fillna(0)
    .reset_index()
)

df_climary_summary

In [ ]:
fig = make_subplots(
    rows=3,
    cols=2,
    subplot_titles=(
        "Avg. Truck Temperature",
        "Avg. Truck Humidity",
        "Avg. Rain",
        "Avg. Snow",
        "Avg. Cloudyness",
        "Avg. Windspeed",
    ),
)

# Adding subplots
fig.add_trace(
    go.Scatter(
        x=df_climary_summary["timestamp_month"],
        y=df_climary_summary["avg_truck_temperature"],
        name="Truck Temperature",
        line_color="#6929c4",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_climary_summary["timestamp_month"],
        y=df_climary_summary["avg_truck_humidity"],
        name="Truck Humidity",
        line_color="#1192e8",
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=df_climary_summary["timestamp_month"],
        y=df_climary_summary["avg_rain"],
        name="Rain",
        line_color="#005d5d",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_climary_summary["timestamp_month"],
        y=df_climary_summary["avg_snow"],
        name="Snow",
        line_color="#9f1853",
    ),
    row=2,
    col=2,
)
fig.add_trace(
    go.Scatter(
        x=df_climary_summary["timestamp_month"],
        y=df_climary_summary["avg_cloudyness"],
        name="Cloudyness",
        line_color="#fa4d56",
    ),
    row=3,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_climary_summary["timestamp_month"],
        y=df_climary_summary["avg_windspeed"],
        name="Windspeed",
        line_color="#570408",
    ),
    row=3,
    col=2,
)

# Remove grid
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# Set x-axis tick angle to be slanted
fig.update_xaxes(tickangle=45)

# Remove background color, set figure size, and title text
fig.update_layout(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    height=800,
    width=1000,
    title_text="Truck Conditions Overview",
    template="plotly_white",
)


fig.show()

# Weather and Climate Correlations

In [ ]:
# Check PK/FK relationship
# There is two PK for every FK because there is value for truck humidity and temperature
df_left[["pkId", "fkSensorSerialId"]].value_counts()

In [ ]:
df_left.dropna(subset=["value"])[
    [
        "timestamp",
        "pkId",
        "fkLinkSerialId",
        "fkReadingsId",
        "fkSensorSerialId",
        "lat",
        "long",
        "weather_station_temperature",
        "weather_station_humidity",
        "rain",
        "snow",
        "country",
        "clouds",
        "wind_speed",
    ]
].isna().sum()

In [ ]:
df_left_pivot = (
    # Temporarily fill NaN values in rain and snow with -1 for pivot
    df_left.fillna({"rain": -1, "snow": -1})
    .pivot_table(
        values="value",
        index=[
            "timestamp",
            "pkId",
            "fkLinkSerialId",
            "fkReadingsId",
            "fkSensorSerialId",
            "lat",
            "long",
            "weather_station_temperature",
            "weather_station_humidity",
            "rain",
            "snow",
            "country",
            "region",
            "clouds",
            "wind_speed",
        ],
        columns="measurement",        
    )
    .rename(columns={"humidity": "truck_humidity", "temperature": "truck_temperature"})
    .reset_index()
    .rename_axis(None, axis=1)
    # Replace values -1 with NaN in column rain and snow after pivot is done
    .replace({"rain": {-1: np.nan}, "snow": {-1: np.nan}})
)

df_left_pivot

In [ ]:
df_corr = df_left_pivot[
    [
        "weather_station_temperature",
        "weather_station_humidity",
        "rain",
        "snow",       
        "clouds",
        "wind_speed",
        "truck_humidity",
        "truck_temperature",
    ]
].corr(method="spearman")
df_corr

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x=df_corr.columns,
        y=df_corr.index,
        z=np.array(df_corr),
        text=df_corr.values,
        texttemplate="%{text:.2f}",
        colorscale=diverging_r_b,
        zmid=0,
    )
)

fig.update_layout(
    title="Correlation Heatmap",
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_tickangle=45,
    width=800,
    height=800,
    template="plotly_white",
)


# Increase font size and remove background color
fig.update_layout(
    font_size=16,
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
)

fig.show()

# Model

In [ ]:
import shap
import catboost

# print the JS visualization code to the notebook
shap.initjs()

In [ ]:
# Pre-process for catboost
df_left_pivot["timestamp_day"] = df_left_pivot["timestamp"].dt.dayofweek.map(day_map)
df_left_pivot["timestamp_day"] = pd.Categorical(
    df_left_pivot["timestamp_day"],
    categories=day_map.values(),
)

df_left_pivot["timestamp_hour"] = df_left_pivot["timestamp"].dt.hour

df_left_pivot["country"] = pd.Categorical(df_left_pivot["country"])

df_left_pivot = df_left_pivot.dropna(subset=["truck_temperature", "truck_humidity"])
df_left_pivot = df_left_pivot.sort_values(by=["timestamp"]).reset_index(drop=True)
df_left_pivot

In [ ]:
features = [
    "lat",
    "long",
    "weather_station_temperature",
    "weather_station_humidity",
    "rain",
    "snow",
    "country",
    "clouds",
    "wind_speed",
    "timestamp_day",
    "timestamp_hour",
]

target_truck_temperature = "truck_temperature"
target_truck_humidity = "truck_humidity"

X = df_left_pivot[features]
y_truck_temperature = df_left_pivot[target_truck_temperature]
y_truck_humidity = df_left_pivot[target_truck_humidity]

In [ ]:
# Two catboost models 
# One for truck temperature and one for truck humidity
model_truck_temp = catboost.CatBoostRegressor(
    iterations=300, learning_rate=0.1, random_seed=123
)
p_temp = catboost.Pool(
    X, y_truck_temperature, cat_features=["country", "timestamp_day"]
)
model_truck_temp.fit(p_temp, verbose=True, plot=False)

model_truck_humidity = catboost.CatBoostRegressor(
    iterations=300, learning_rate=0.1, random_seed=123
)
p_humidity = catboost.Pool(
    X, y_truck_humidity, cat_features=["country", "timestamp_day"]
)
model_truck_humidity.fit(p_humidity, verbose=True, plot=False)

# Feature Importance & Explainability

## Truck Temperature

In [ ]:
# Explain predictions for the first observation for predicting truck temperature
explainer = shap.TreeExplainer(model_truck_temp)
shap_values = explainer.shap_values(X)

# Visualize the first prediction's explanation
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
# Number of samples to take from the data; too large values will run out of memory
sample_size = 500
X_sample = X.sample(sample_size, random_state=123)

# Recalculate shap_values for subsampled data
shap_values_sample = explainer.shap_values(X_sample)

# Visualize using the subsampled data
shap.force_plot(explainer.expected_value, shap_values_sample, X_sample)

## Truck Humidity

In [ ]:
# Explain predictions for the first observation for predicting truck humidity
explainer = shap.TreeExplainer(model_truck_humidity)
shap_values = explainer.shap_values(X)

# Visualize the first prediction's explanation
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
# Number of samples to take from the data; too large values will run out of memory
sample_size = 500
X_sample = X.sample(sample_size, random_state=123)

# Recalculate shap_values for subsampled data
shap_values_sample = explainer.shap_values(X_sample)

# Visualize using the subsampled data
shap.force_plot(explainer.expected_value, shap_values_sample, X_sample)

# Simulated Trip

In [ ]:
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
from math import radians, cos, sin, asin, sqrt
from datetime import datetime, timedelta, timezone

In [ ]:
owm = OWM(OPEN_WEATHER_API_KEY)
mgr = owm.weather_manager()

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r


def get_spaced_points(lat1, long1, lat2, long2, num_points):
    """
    Return a list of latitudes and longitudes num_points apart between start and end points
    """
    lats = np.linspace(lat1, lat2, num_points)
    longs = np.linspace(long1, long2, num_points)
    return lats, longs

def openweather_one_hour_history(lat, lon, dt):
    forecast_time = mgr.one_call_history(
        lat=float(lat),
        lon=float(lon),
        dt=int(dt),
    )

    list_of_forecasted_weathers = forecast_time.forecast_hourly
    dt_iso = pd.to_datetime(dt, unit="s", utc=True)

    for forecast in list_of_forecasted_weathers:
        forecast_time_iso = pd.to_datetime(forecast.reference_time("iso"))

        # print(forecast.reference_time("iso"), dt_iso)
        if forecast_time_iso == dt_iso:
            date = forecast.reference_time()
            temp = forecast.temperature("celsius").get("temp")
            rain = forecast.rain
            snow = forecast.snow
            clouds = forecast.clouds
            wind = forecast.wind().get("speed")
            humidity = forecast.humidity
            # print(date, temp, rain, clouds, wind, humidity)
            return {
                "timestamp": date,
                "temp": temp,
                "rain": rain,
                "snow": snow,
                "clouds": clouds,
                "wind_speed": wind,
                "humidity": humidity,
            }

In [ ]:
kmph = 45 * 1.609
start_lat, start_long = 43.481777, -81.069008
end_lat, end_long = 45.482050, -73.792810

In [ ]:
distance = haversine(start_long, start_lat, end_long, end_lat)
duration = distance / kmph
num_points = int(distance / kmph)  # This will give the number of points based on speed
lats, longs = get_spaced_points(start_lat, start_long, end_lat, end_long, num_points)

print(
    f"Distance between {start_lat}, {start_long} and {end_lat}, {end_long} is {distance} km"
)
print(f"Duration is ~{num_points} hours if speed is {kmph} kmph")

In [ ]:
simulated_trip = pd.DataFrame(
    {
        "lat": lats,
        "long": longs,
    }
)

# OpenWeather allows a 5 days history. 
start = "2023-07-01"
simulated_trip["timestamp"] = pd.date_range(
    start=start, periods=len(simulated_trip), freq="H"
)

simulated_trip["epochs"] = simulated_trip["timestamp"].apply(
    lambda x: int(x.replace(tzinfo=timezone.utc).timestamp())
)

forecast_list = []
for index, row in simulated_trip.iterrows():
    forecast_list.append(openweather_one_hour_history(row["lat"], row["long"], row["epochs"]))

forecast_df = pd.DataFrame.from_dict(forecast_list)
forecast_df

In [ ]:
# Format feature columns to match the model
sim_df = (
    pd.merge(simulated_trip, forecast_df, left_on="epochs", right_on="timestamp")
    .drop(columns=["epochs", "timestamp_y"])
    .rename(columns={"timestamp_x": "timestamp"})
    .rename(
        columns={
            "temp": "weather_station_temperature",
            "humidity": "weather_station_humidity",
        }
    )
)
sim_df["country"] = "ca"
sim_df["timestamp_day"] = sim_df["timestamp"].dt.dayofweek.map(day_map)
sim_df["timestamp_day"] = pd.Categorical(
    sim_df["timestamp_day"],
    categories=day_map.values(),
)
sim_df["timestamp_hour"] = sim_df["timestamp"].dt.hour

sim_df = sim_df.drop(columns=["timestamp"])

# Replace {} values with nan in rain and snow column
sim_df["rain"] = np.nan
sim_df["snow"] = np.nan

sim_df = sim_df[
    [
        "lat",
        "long",
        "weather_station_temperature",
        "weather_station_humidity",
        "rain",
        "snow",
        "country",
        "clouds",
        "wind_speed",
        "timestamp_day",
        "timestamp_hour",
    ]
]

sim_df

| lat | long | weather_station_temperature | weather_station_humidity | rain | snow | country | clouds | wind_speed | timestamp_day | timestamp_hour |
|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|
| 43.481777 | -81.069008 | 22.93 | 85 | NaN | NaN | ca | 59 | 2.93 | Saturday | 0 |
| 43.767530 | -80.029551 | 18.94 | 92 | NaN | NaN | ca | 91 | 2.56 | Saturday | 1 |
| 44.053284 | -78.990094 | 17.64 | 73 | NaN | NaN | ca | 4 | 1.68 | Saturday | 2 |
| 44.339037 | -77.950637 | 17.70 | 84 | NaN | NaN | ca | 1 | 2.04 | Saturday | 3 |
| 44.624790 | -76.911181 | 16.85 | 87 | NaN | NaN | ca | 6 | 1.98 | Saturday | 4 |
| 44.910543 | -75.871724 | 16.91 | 84 | NaN | NaN | ca | 3 | 2.50 | Saturday | 5 |
| 45.196297 | -74.832267 | 17.33 | 89 | NaN | NaN | ca | 2 | 2.45 | Saturday | 6 |
| 45.482050 | -73.792810 | 18.22 | 93 | NaN | NaN | ca | 4 | 1.43 | Saturday | 7 |

In [ ]:
# Predict with both catboost models
sim_df[target_truck_temperature] = model_truck_temp.predict(sim_df)
sim_df[target_truck_humidity] = model_truck_humidity.predict(sim_df)
sim_df

| lat | long | weather_station_temperature | weather_station_humidity | rain | snow | country | clouds | wind_speed | timestamp_day | timestamp_hour | truck_temperature | truck_humidity |
|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|
| 43.481777 | -81.069008 | 22.93 | 85 | NaN | NaN | ca | 59 | 2.93 | Saturday | 0 | 23.063771 | 96.077573 |
| 43.767530 | -80.029551 | 18.94 | 92 | NaN | NaN | ca | 91 | 2.56 | Saturday | 1 | 22.483871 | 91.254676 |
| 44.053284 | -78.990094 | 17.64 | 73 | NaN | NaN | ca | 4 | 1.68 | Saturday | 2 | 22.516637 | 68.043051 |
| 44.339037 | -77.950637 | 17.70 | 84 | NaN | NaN | ca | 1 | 2.04 | Saturday | 3 | 21.647016 | 76.923307 |
| 44.624790 | -76.911181 | 16.85 | 87 | NaN | NaN | ca | 6 | 1.98 | Saturday | 4 | 20.662621 | 80.147000 |
| 44.910543 | -75.871724 | 16.91 | 84 | NaN | NaN | ca | 3 | 2.50 | Saturday | 5 | 21.454387 | 77.273963 |
| 45.196297 | -74.832267 | 17.33 | 89 | NaN | NaN | ca | 2 | 2.45 | Saturday | 6 | 22.348031 | 79.396023 |
| 45.482050 | -73.792810 | 18.22 | 93 | NaN | NaN | ca | 4 | 1.43 | Saturday | 7 | 24.426549 | 81.435330 |

# Visualize Trip & Predictions

In [ ]:
# Hard coded values for lat and long due to historical limitation of OpenWeather
df = pd.DataFrame(
    {
        "lat": [
            43.481777,
            43.767530,
            44.053284,
            44.339037,
            44.624790,
            44.910543,
            45.196297,
            45.482050,
        ],
        "long": [
            -81.069008,
            -80.029551,
            -78.990094,
            -77.950637,
            -76.911181,
            -75.871724,
            -74.832267,
            -73.792810,
        ],
        "truck_temperature": [
            23.063771,
            22.483871,
            22.516637,
            21.647016,
            20.662621,
            21.454387,
            22.348031,
            24.426549,
        ],
        "truck_humidity": [
            96.077573,
            91.254676,
            68.043051,
            76.923307,
            80.147000,
            77.273963,
            79.396023,
            81.435330,
        ],
    }
)

# Create a scatter mapbox plot for the route
route = go.Scattermapbox(
    lat=df["lat"],
    lon=df["long"],
    mode="markers+lines+text",
    marker=go.scattermapbox.Marker(size=9),
    text=[
        f"Temp: {temp:.2f}°C Humidity: {humid:.2f}%"
        for temp, humid in zip(df["truck_temperature"], df["truck_humidity"])
    ],
    textposition="bottom right",
)

# Create scatter mapbox plot for the start point
start = go.Scattermapbox(
    lat=[df["lat"].iloc[0]],
    lon=[df["long"].iloc[0]],
    mode="markers+text",
    marker=go.scattermapbox.Marker(size=12, symbol="car"),
    text=f"Start<br>Temp: {df['truck_temperature'].iloc[0]:.2f}°C<br>Humidity: {df['truck_humidity'].iloc[0]:.2f}%",
    textposition="bottom left",
)


# Create scatter mapbox plot for the end point
end = go.Scattermapbox(
    lat=[df["lat"].iloc[-1]],
    lon=[df["long"].iloc[-1]],
    mode="markers+text",
    marker=go.scattermapbox.Marker(size=12, symbol="star"),
    text=f"End<br>Temp: {df['truck_temperature'].iloc[-1]:.2f}°C<br>Humidity: {df['truck_humidity'].iloc[-1]:.2f}%",
    textposition="top right",
)

# Update layout
fig = go.Figure(data=[route, start, end])

# Update layout
fig.update_layout(
    autosize=True,
    hovermode="closest",
    mapbox=dict(
        accesstoken=MAPBOX_API_KEY,
        bearing=0,
        center=dict(lat=df["lat"].mean(), lon=df["long"].mean()),
        pitch=0,
        zoom=6.2,
        style="light",
    ),
)


# Fig layout
fig.update_layout(
    height=800,
    width=1200,
    title_text="Simulated Route and Internal Truck Conditions",
    template="plotly_white",
    showlegend=False,
)

fig.show()

# Forecasting Model

In [ ]:
df_left_pivot["timestamp_year"] = df_left_pivot["timestamp"].dt.year
df_left_pivot["timestamp_month"] = df_left_pivot["timestamp"].dt.month
df_left_pivot["timestamp_day"] = df_left_pivot["timestamp"].dt.day

# Country and region to filter
country = "ca"
region = "CA-ON"

df_left_time_series = (
    df_left_pivot.query(f"country == '{country}' and region == '{region}'")
    .groupby(
        [
            "country",
            "region",
            "timestamp_year",
            "timestamp_month",
            "timestamp_day",
            "timestamp_hour",
        ]
    )
    .agg(
        weather_station_temperature=("weather_station_temperature", "mean"),
        weather_station_humidity=("weather_station_humidity", "mean"),
        rain=("rain", "mean"),
        snow=("snow", "mean"),
        clouds=("clouds", "mean"),
        wind_speed=("wind_speed", "mean"),
        truck_humidity=("truck_humidity", "mean"),
        truck_temperature=("truck_temperature", "mean"),
    )
    .dropna(subset=["truck_humidity"])
    .reset_index()
)

df_left_time_series

In [ ]:
ts_map = {
    "timestamp_year": "year",
    "timestamp_month": "month",
    "timestamp_day": "day",
    "timestamp_hour": "hour",
}

df_left_time_series["timestamp"] = pd.to_datetime(
    df_left_time_series.rename(columns=ts_map)[["year", "month", "day", "hour"]]
)

df_left_time_series = df_left_time_series.set_index("timestamp")

# Resample to hourly data, not the greatest way to handle this
df_hourly = df_left_time_series.resample("H").interpolate(method="ffill").reset_index()

df_hourly


In [ ]:
from sktime.regression.deep_learning import CNNRegressor
from sktime.forecasting.compose import make_reduction


# `make_reduction` automatically handles creation of lagged features and other boilerplate code for timeseries forecasting
forecaster_internal_truck_temp = make_reduction(
    estimator=CNNRegressor(
        n_epochs=10
    ),  # set epochs to 10 for quick training
    window_length=12,
    strategy="recursive",
)

y = df_hourly[["timestamp", "truck_temperature", "truck_humidity"]].set_index(
    "timestamp"
)
forecaster_internal_truck_temp.fit(y)

In [ ]:
# Predict three days in the future
days = 3
fh = np.arange(1, 24 * days + 1)
predictions = forecaster_internal_truck_temp.predict(fh)

In [ ]:
fig = px.line(
    predictions.reset_index(),
    x="timestamp",
    y=["truck_temperature", "truck_humidity"],
    template="plotly_white",
    title=f"Truck Temperature & Humidity, {days} Day Forecast, {region}",
    color_discrete_sequence=["#6929c4", "#1192e8"],
)

# Remove grid
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# Remove background color, and update figure size.
fig.update_layout(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    height=600,
    width=1000,
)


fig.show()